<a href="https://colab.research.google.com/github/AlexMontgomerie/deepLearning/blob/master/denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Denoise Network

## Google Setup

In [2]:
!git clone https://github.com/AlexMontgomerie/deepLearning
%cd deepLearning
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

fatal: destination path 'deepLearning' already exists and is not an empty directory.
/content/deepLearning
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 6, in <module>
    from pip._internal import main
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/__init__.py", line 40, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/autocompletion.py", line 8, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/cmdoptions.py", line 22, in <module>
    from pip._internal.utils.hashes import STRONG_HASHES
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/utils/hashes.py", line 10, in <module>
    from pip._internal.utils.misc impor

## Setup

In [3]:
from common import *
!./setup.sh

Using TensorFlow backend.


## Get Denoise Model (Add)

In [0]:
from denoise_network import *
from get_data import get_data

EPOCHS = 50

seqs_train, seqs_test = get_data()

# get traning data
if os.path.exists('data/denoise_data/denoise_generator.npy') and os.path.exists('data/denoise_data/denoise_generator_val.npy'):
    denoise_generator     = np.load('data/denoise_data/denoise_generator.npy'    )
    denoise_generator_val = np.load('data/denoise_data/denoise_generator_val.npy')

else:
    denoise_generator     = DenoiseHPatches(seqs_train, batch_size=500)
    denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=500)
    np.save('data/denoise_data/denoise_generator.npy'    , denoise_generator    )
    np.save('data/denoise_data/denoise_generator_val.npy', denoise_generator_val)


# get model
shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)

# callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 3, mode= 'auto'),
    keras.callbacks.ModelCheckpoint('data/denoise_model.weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
]

# optimiser
opt  = opt = keras.optimizers.nadam()

# loss
loss = 'mean_absolute_error'

# train network
denoise_model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
denoise_history = denoise_model.fit_generator(generator=denoise_generator, epochs=EPOCHS, callbacks=callbacks,
                                              verbose=1, validation_data=denoise_generator_val)

  3%|▎         | 2/76 [00:02<01:32,  1.25s/it]